In [1]:
import numpy as np
from jass.game.const import card_strings
from jass.game.game_sim import GameSim
from jass.game.rule_schieber import RuleSchieber

from play_rule_strategy.mini_max.mini_maxer import MiniMaxer
from play_strategy.random_play_strategy import RandomPlayStrategy
from trump_strategy.deep_nn_trump_strategy import DeepNNTrumpStrategy

In [2]:
hands = np.array([
    [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1]
])

In [3]:
from jass.game.const import trump_strings_german_long

game_sim = GameSim(RuleSchieber())
# dealer = random.randint(0, 3)
dealer = 0

trump_strategy = DeepNNTrumpStrategy("../data/deep_nn_trump_selector.pt")
random_play = RandomPlayStrategy(seed=43)

game_sim.init_from_cards(hands, dealer)

# Disable PUSH to simplify the game
obs = game_sim.get_observation()
obs.forehand = 1

best_trump = trump_strategy.choose_trump(obs)
game_sim.action_trump(best_trump)
n_player_cards = 2
while (game_sim.get_observation().player != 0
       or game_sim.state.hands[0].sum() > n_player_cards):
    game_sim.action_play_card(random_play.choose_card(game_sim.get_observation()), )

# player one hand cast to string
print(f"trump: {trump_strings_german_long[game_sim.state.trump]}")

current_trick_string = [card_strings[i] if i != -1 else "None" for i in game_sim.state.current_trick]
print(f"Current Trick: {current_trick_string}")

print(f"Player 1 hand: {[card_strings[i] for i in np.where(game_sim.state.hands[0] == 1)[0]]}")
print(f"Player 2 hand: {[card_strings[i] for i in np.where(game_sim.state.hands[1] == 1)[0]]}")
print(f"Player 3 hand: {[card_strings[i] for i in np.where(game_sim.state.hands[2] == 1)[0]]}")
print(f"Player 4 hand: {[card_strings[i] for i in np.where(game_sim.state.hands[3] == 1)[0]]}")

print((game_sim.get_observation().points[0] - game_sim.get_observation().points[1]) / 157)

trump: U: Une-Ufe
Current Trick: ['S9', 'None', 'None', 'None']
Player 1 hand: ['HK', 'H8']
Player 2 hand: ['HJ']
Player 3 hand: ['H6', 'CQ']
Player 4 hand: ['H9', 'C7']
-0.5732484076433121


In [4]:

mini_maxer = MiniMaxer()

root = mini_maxer.search(game_sim.state, 10)

mini_maxer.print_tree(root)

 ROOT: -0.45222929936305734
     HK: -0.45222929936305734
         H9: -0.45222929936305734
             H6: -0.554140127388535
                 HJ: -0.554140127388535
                     H8: -0.554140127388535
                         C7: -0.554140127388535
                             CQ: 0
             CQ: -0.45222929936305734
                 HJ: -0.45222929936305734
                     H8: -0.45222929936305734
                         C7: -0.45222929936305734
                             H6: 0
         C7: -0.45222929936305734
             H6: -0.554140127388535
                 HJ: -0.554140127388535
                     H8: -0.554140127388535
                         H9: -0.554140127388535
                             CQ: 0
             CQ: -0.45222929936305734
                 HJ: -0.45222929936305734
                     H8: -0.45222929936305734
                         H9: -0.45222929936305734
                             H6: 0
     H8: -0.5031847133757962
         H9: -0.5